In [26]:
pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 82.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.4/308.4 kB 21.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [28]:
!pip install eofs

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.2 MB/s eta 0:00:00a 0:00:01


In [30]:
!pip install utils

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Created wheel for utils: filename=utils-1.0.2-py2.py3-none-any.whl size=13906 sha256=848ab2eb0245d2a4752d54f4aa40ce6d1ee553fb3c1b85fdeeab7eb4dd335aa2
  Stored in directory: /glade/u/home/sivaram/.cache/pip/wheels/b8/39/f5/9d0ca31dba85773ececf0a7f5469f18810e1c8a8ed9da28ca7
Successfully built utils


In [31]:
import os
import datetime as dt  # Python standard library datetime  module
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import netCDF4 as nc
import xarray as xr
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import RFE
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor 
from sklearn.model_selection import RandomizedSearchCV
from eofs.xarray import Eof
import glob
from utils import data_path

ImportError: cannot import name 'data_path' from 'utils' (/glade/u/home/sivaram/.local/lib/python3.10/site-packages/utils/__init__.py)

In [2]:
 !wget https://zenodo.org/record/7064308/files/train_val.tar.gz

--2025-11-11 15:58:17--  https://zenodo.org/record/7064308/files/train_val.tar.gz
Resolving zenodo.org (zenodo.org)... 188.185.48.75, 137.138.52.235, 188.185.43.153, ...
Connecting to zenodo.org (zenodo.org)|188.185.48.75|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/7064308/files/train_val.tar.gz [following]
--2025-11-11 15:58:18--  https://zenodo.org/records/7064308/files/train_val.tar.gz
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 839129430 (800M) [application/octet-stream]
Saving to: ‘train_val.tar.gz.1’

train_val.tar.gz.1  100%[===================>] 800.26M  25.2MB/s    in 35s     

2025-11-11 15:58:53 (22.7 MB/s) - ‘train_val.tar.gz.1’ saved [839129430/839129430]



In [6]:
!wget -c https://zenodo.org/record/7064308/files/test.tar.gz

--2025-11-11 21:09:36--  https://zenodo.org/record/7064308/files/test.tar.gz
Resolving zenodo.org (zenodo.org)... 137.138.52.235, 188.185.48.75, 188.185.43.153, ...
Connecting to zenodo.org (zenodo.org)|137.138.52.235|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/7064308/files/test.tar.gz [following]
--2025-11-11 21:09:37--  https://zenodo.org/records/7064308/files/test.tar.gz
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 74442376 (71M) [application/octet-stream]
Saving to: ‘test.tar.gz’

test.tar.gz         100%[===================>]  70.99M  3.47MB/s    in 15s     

2025-11-11 21:09:52 (4.60 MB/s) - ‘test.tar.gz’ saved [74442376/74442376]



In [1]:
!wget -c https://zenodo.org/record/7064308/files/CMIP6.zip

--2025-11-11 20:54:20--  https://zenodo.org/record/7064308/files/CMIP6.zip
Resolving zenodo.org (zenodo.org)... 188.185.43.153, 137.138.52.235, 188.185.48.75, ...
Connecting to zenodo.org (zenodo.org)|188.185.43.153|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/7064308/files/CMIP6.zip [following]
--2025-11-11 20:54:22--  https://zenodo.org/records/7064308/files/CMIP6.zip
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 1473338296 (1.4G) [application/octet-stream]
Saving to: ‘CMIP6.zip’

CMIP6.zip           100%[===================>]   1.37G  3.53MB/s    in 7m 21s  

2025-11-11 21:01:43 (3.19 MB/s) - ‘CMIP6.zip’ saved [1473338296/1473338296]



In [8]:
!tar -xzf train_val.tar.gz

In [52]:
!tar -xzf test.tar.gz -C test_val

tar: test_val: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [39]:
train_files_input = glob.glob("train_val/inputs*.nc")
train_files_input

['train_val/inputs_1pctCO2.nc',
 'train_val/inputs_abrupt-4xCO2.nc',
 'train_val/inputs_ssp126.nc',
 'train_val/inputs_ssp370-lowNTCF.nc',
 'train_val/inputs_hist-GHG.nc',
 'train_val/inputs_historical.nc',
 'train_val/inputs_hist-aer.nc',
 'train_val/inputs_ssp370.nc',
 'train_val/inputs_ssp585.nc']

In [41]:
data_path = "./train_val/"
X_historical = xr.open_dataset(data_path + "inputs_historical.nc")


In [49]:
test_sets = [
    "historical",
    "ssp585",
    "ssp126",
    "ssp370",
    "hist-aer",
    "hist-GHG"
]


In [46]:
test_hist_co2 = normalize_co2(X_historical["CO2"].data)
test_hist_ch4 = normalize_ch4(X_historical["CH4"].data)

In [40]:
min_co2 = 0.
max_co2 = 9500
def normalize_co2(data):
    return data / max_co2

def un_normalize_co2(data):
    return data * max_co2

min_ch4 = 0.
max_ch4 = 0.8
def normalize_ch4(data):
    return data / max_ch4

def un_normalize_ch4(data):
    return data * max_ch4

In [47]:
def create_predictor_data(data_sets, n_eofs=5):
    """
    Args:
        data_sets list(str): names of datasets
        n_eofs (int): number of eofs to create for aerosol variables
    """
        
    # Create training and testing arrays
    if isinstance(data_sets, str):
        data_sets = [data_sets]
    X = xr.concat([xr.open_dataset(data_path + f"inputs_{file}.nc") for file in data_sets], dim='time')
    X = X.assign_coords(time=np.arange(len(X.time)))

    # Compute EOFs for BC
    bc_solver = Eof(X['BC'])
    bc_eofs = bc_solver.eofsAsCorrelation(neofs=n_eofs)
    bc_pcs = bc_solver.pcs(npcs=n_eofs, pcscaling=1)

    # Compute EOFs for SO2
    so2_solver = Eof(X['SO2'])
    so2_eofs = so2_solver.eofsAsCorrelation(neofs=n_eofs)
    so2_pcs = so2_solver.pcs(npcs=n_eofs, pcscaling=1)

    # Convert to pandas
    bc_df = bc_pcs.to_dataframe().unstack('mode')
    bc_df.columns = [f"BC_{i}" for i in range(n_eofs)]

    so2_df = so2_pcs.to_dataframe().unstack('mode')
    so2_df.columns = [f"SO2_{i}" for i in range(n_eofs)]

    # Bring the emissions data back together again and normalise
    inputs = pd.DataFrame({
        "CO2": normalize_co2(X["CO2"].data),
        "CH4": normalize_ch4(X["CH4"].data)
    }, index=X["CO2"].coords['time'].data)

    # Combine with aerosol EOFs
    inputs = pd.concat([inputs, bc_df, so2_df], axis=1)
    return inputs, (so2_solver, bc_solver)

In [84]:
X_train, train_eof = create_predictor_data(test_sets)

In [86]:
train_eof

(<eofs.xarray.Eof at 0x1530b5e16590>, <eofs.xarray.Eof at 0x1530c0a337c0>)

In [91]:
train_eof[0]

In [118]:
X_test

,CO2,CH4,BC_0,BC_1,BC_2,BC_3,BC_4,SO2_0,SO2_1,SO2_2,SO2_3,SO2_4
2015,0.161692,0.467171,2.706810,-1.375172,-0.819380,-0.256301,-1.716714,0.903505,2.331736,-0.104266,-1.138111,1.045259
2016,0.165480,0.467151,2.648897,-1.303500,-0.707064,-0.314685,-1.616016,0.857117,2.261433,0.078151,-1.083139,0.977491
2017,0.169305,0.467130,2.590985,-1.231828,-0.594749,-0.373069,-1.515319,0.810728,2.191129,0.260569,-1.028166,0.909724
2018,0.173166,0.467110,2.533072,-1.160157,-0.482433,-0.431453,-1.414621,0.764340,2.120826,0.442987,-0.973193,0.841956
2019,0.177064,0.467090,2.475159,-1.088485,-0.370117,-0.489837,-1.313923,0.717951,2.050522,0.625405,-0.918221,0.774188
...,...,...,...,...,...,...,...,...,...,...,...,...
2096,0.469755,0.352172,0.434483,-0.189906,-0.115759,-0.230166,0.147331,0.178097,0.697813,1.189839,-0.064685,0.033707
2097,0.471474,0.351729,0.422541,-0.181722,-0.101978,-0.213992,0.169751,0.177501,0.687329,1.167448,-0.080802,0.006906
2098,0.473128,0.351287,0.410599,-0.173539,-0.088197,-0.197818,0.192172,0.176905,0.676844,1.145056,-0.096920,-0.019895
2099,0.474718,0.350845,0.398656,-0.165355,-0.074416,-0.181644,0.214592,0.176309,0.666359,1.122665,-0.113037,-0.046696


In [53]:
def create_predictdand_data(data_sets):
    if isinstance(data_sets, str):
        data_sets = [data_sets]
    Y = xr.concat([xr.open_dataset(data_path + f"outputs_{file}.nc") for file in data_sets], dim='time').mean("member")
    # Convert the precip values to mm/day
    Y["pr"] *= 86400
    Y["pr90"] *= 86400
    return Y

In [57]:
Y_train = create_predictdand_data(test_sets)

In [121]:
Y_train

<xarray.Dataset> Size: 208MB
Dimensions:                    (time: 753, lat: 96, lon: 144)
Coordinates:
  * lat                        (lat) float64 768B -90.0 -88.11 ... 88.11 90.0
  * lon                        (lon) float64 1kB 0.0 2.5 5.0 ... 355.0 357.5
  * time                       (time) int64 6kB 1850 1851 1852 ... 2013 2014
    quantile                   float64 8B 0.9
Data variables:
    diurnal_temperature_range  (time, lat, lon) float32 42MB 0.03264 ... -0.3354
    tas                        (time, lat, lon) float32 42MB 0.32 ... 2.611
    pr                         (time, lat, lon) float32 42MB 0.02337 ... 0.04197
    pr90                       (time, lat, lon) float64 83MB 0.02468 ... 0.08654

In [123]:
y_train_tas = Y_train["tas"].mean(["lat","lon"]).data
y_train_pr = Y_train["pr"].mean(["lat","lon"]).data
y_train_pr90 = Y_train["pr90"].mean(["lat","lon"]).data
y_train_DTR = Y_train["diurnal_temperature_range"].mean(["lat","lon"]).data

In [61]:
rf = RandomForestRegressor(
    n_estimators= 100,
    random_state=  50,
    n_jobs = -1
)

In [65]:
rf.fit(X_train,y_train_tas)

,n_estimators,100
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [67]:
!mkdir -p test_data
!tar -xzf test.tar.gz -C test_data


In [93]:
def get_test_data(file, eof_solvers, n_eofs=5):
    """
    Args:
        file str: name of datasets
        n_eofs (int): number of eofs to create for aerosol variables
        eof_solvers (Eof_so2, Eof_bc): Fitted Eof objects to use for projection
    """
    data_path_1 = "./test_data/"    
    # Create training and testing arrays
    X = xr.open_dataset(data_path_1 + f"inputs_{file}.nc")
        
    so2_pcs = eof_solvers[0].projectField(X["SO2"], neofs=5, eofscaling=1)
    so2_df = so2_pcs.to_dataframe().unstack('mode')
    so2_df.columns = [f"SO2_{i}" for i in range(n_eofs)]

    bc_pcs = eof_solvers[1].projectField(X["BC"], neofs=5, eofscaling=1)
    bc_df = bc_pcs.to_dataframe().unstack('mode')
    bc_df.columns = [f"BC_{i}" for i in range(n_eofs)]

    # Bring the emissions data back together again and normalise
    inputs = pd.DataFrame({
        "CO2": normalize_co2(X["CO2"].data),
        "CH4": normalize_ch4(X["CH4"].data)
    }, index=X["CO2"].coords['time'].data)

    # Combine with aerosol EOFs
    inputs = pd.concat([inputs, bc_df, so2_df], axis=1)
    return inputs

In [95]:
def create_predictdand_data_test(data_sets):
    data_path = "./test_data/"    
    if isinstance(data_sets, str):
        data_sets = [data_sets]
    Y = xr.concat([xr.open_dataset(data_path + f"outputs_{file}.nc") for file in data_sets], dim='time').mean("member")
    # Convert the precip values to mm/day
    Y["pr"] *= 86400
    Y["pr90"] *= 86400
    return Y

In [113]:
X_test = get_test_data("ssp245",train_eof)
Y_test = create_predictdand_data_test("ssp245")

In [114]:
y_tas_pred = rf.predict(X_test)
len(y_tas_pred)

86

In [126]:
def get_rmse(truth, pred):
    """
    Compute RMSE between true values and predictions.

    Args:
        truth (np.ndarray): Ground truth values
        pred (np.ndarray): Predicted values

    Returns:
        float: RMSE
    """
    return np.sqrt(np.mean((truth - pred)**2))


In [129]:
Y_test_tas = Y_test["tas"].mean(["lat","lon"]).data
Y_test_pr = Y_test["pr"].mean(["lat","lon"]).data
Y_test_pr90 = Y_test["pr90"].mean(["lat","lon"]).data
Y_test_DTR = Y_test["diurnal_temperature_range"].mean(["lat","lon"]).data




In [127]:
pr = rf.fit(X_train,y_train_pr)
y_pr_pred = pr.predict(X_test)
get_rmse(Y_test_pr,y_pr_pred)

0.015105975959860851

In [130]:
pr90 = rf.fit(X_train,y_train_pr90)
y_pr90_pred = pr.predict(X_test)
get_rmse(Y_test_pr90,y_pr90_pred)

0.0399722318323548

In [131]:
DTR = rf.fit(X_train,y_train_DTR)
y_DTR_pred = pr.predict(X_test)
get_rmse(Y_test_DTR,y_DTR_pred)

0.019705957995763305

In [117]:
get_rmse(Y_test_tas,y_tas_pred)

0.2900129960834046